Import necessary libraries, initialize the starting year and ending year. 
Select columns that you want to read in.
Select the desired airlines via codes. This can be found on The  Bureau of Transportation's website 
Start the timer and initialize an empty list where the dataframes will be stored.

In [ ]:
import pandas as pd
import time
import os
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

start_year = 2013
end_year = 2023

selected_columns = ['Year', 'Month', 'DayofMonth', 'DayOfWeek', 'DOT_ID_Reporting_Airline',
                     'OriginAirportID', 'OriginCityName',
                    'DestAirportID','DestCityName', 'CRSDepTime', 'DepTime', 'DepDelayMinutes',
                     'TaxiOut', 'TaxiIn', 'CRSArrTime', 'ArrTime', 'ArrDelayMinutes',
                     'Cancelled', 'CancellationCode', 'Diverted', 'CRSElapsedTime',
                     'ActualElapsedTime', 'AirTime', 'Distance', 'CarrierDelay', 'WeatherDelay',
                     'NASDelay', 'SecurityDelay', 'LateAircraftDelay']

Top10AirportCodes = [11298, 12892, 10397, 11292, 12266, 12264, 13204, 13930, 12478, 14747]

start_time = time.time()

data_frames = []

A for loop then reads in the files from my computer and combines them into a single dataframe.
    Note: the year 2023 only has 8 months of data.
Prints an exception if a file was not downloaded properly. Only reads in the rows where the column "OriginAirportID" has a value in the Top10AirportCodes. Append dataframe to the list. Documents time and lets me know when the program is complete. Outputs to the file (called 10Airlines.csv) in the folder airline_data. 

In [ ]:
for year in range(start_year, end_year + 1):
    max_month = 12 if year != 2023 else 8
    for month in range(1, max_month + 1):
        file_path = f"airline_data/{year}/On_Time_Reporting_Carrier_On_Time_Performance_1987_present_{year}_{month}/On_Time_Reporting_Carrier_On_Time_Performance_(1987_present)_{year}_{month}.csv"
        try:
            df = pd.read_csv(file_path, usecols=selected_columns)
            df = df[df['OriginAirportID'].isin(Top10AirportCodes) & df['DestAirportID'].isin(Top10AirportCodes)]
            data_frames.append(df)

        except FileNotFoundError: 
            print(f"File not found for {year}-{month}")

final_df = pd.concat(data_frames, ignore_index=True)

end_time = time.time()
execution_time = end_time - start_time
print(f"Execution time: {execution_time} seconds")

output_dir = 'airline_data'
os.makedirs(output_dir, exist_ok=True)

final_df.to_csv(os.path.join(output_dir, '10Airlines.csv'), index=False)

Reads in the DataFrame and Foreign Keys which map codes to strings. Cleans data and missing values 
I drop the cancellation rows as this would confuse the machine learning model for predicing delays. 

In [ ]:
df = pd.read_csv('airline_data/10Airlines.csv')

#Cleaning CancellationCodes
df["CancellationReason"] = df["CancellationCode"].replace({'A':"Carrier",'B':"Weather",'C':"National Air System",'D':"Security",np.nan:'No Delay'})
df.drop(columns=['CancellationCode'],inplace=True)

#Cleaning Airline name 
DOT_ID_Airline_df = pd.read_csv('airline_data/L_AIRLINE_ID.csv')
ID_map_dict = dict(zip(DOT_ID_Airline_df['Code'], DOT_ID_Airline_df['Description'].str.split(':').str[0])) 
df['Airline'] = df['DOT_ID_Reporting_Airline'].map(ID_map_dict)
df.drop(columns=['DOT_ID_Reporting_Airline'],inplace=True)
#Fix this outlier
df['Airline'] = df['Airline'].replace('ExpressJet Airlines LLC d/b/a aha!', 'ExpressJet Airlines')

#Cleaning Delays (Some airlines did NaN on delays when they didn't exist. These are equivalent to a 0 time delay.)
fill_columns = ['CarrierDelay','WeatherDelay','NASDelay','SecurityDelay','LateAircraftDelay']
df[fill_columns] = df[fill_columns].fillna(0)

#Cleaning DayOfWeek 
DayOfWeek_mapping = {1:'Mon',2:'Tues',3:'Wed',4:'Thurs',5:'Fri',6:'Sat',7:'Sun'}
df['DayOfWeek'] = df['DayOfWeek'].map(DayOfWeek_mapping)

#Cleaning OriginAirportID
ForeignKey = pd.read_csv('airline_data/L_AIRPORT_ID.csv')
ForeignKeyDict = dict(zip(ForeignKey['Code'], ForeignKey['Description']))
df['OriginAirport'] = df['OriginAirportID'].map(ForeignKeyDict)
df.drop(columns=['OriginAirportID'],inplace=True)

#Cleaning Months
ForeignKey = pd.read_csv('airline_data/L_MONTHS.csv')
ForeignKeyDict = dict(zip(ForeignKey['Code'], ForeignKey['Description']))
df['Month'] = df['Month'].map(ForeignKeyDict)

#Cleaning DestAirportID
ForeignKey = pd.read_csv('airline_data/L_AIRPORT_ID.csv')
ForeignKeyDict = dict(zip(ForeignKey['Code'], ForeignKey['Description']))
df['DestAirport'] = df['DestAirportID'].map(ForeignKeyDict)
df.drop(columns=['DestAirportID'],inplace=True)

#Dropping Canceled Rows for Regression Model
df = df[df['Cancelled'] == 0]

2% of rows have NaN values. I looked for a correlation between these rows and none existed. This should not affect our data. We will drop these rows. 

In [ ]:
df.dropna(inplace=True)

A brief look at correlations via a correlation matrix for numeric columns. 

In [ ]:
target_columns = ['ArrDelayMinutes', 'DepDelayMinutes']

numeric_columns = df.select_dtypes(include=['number']).columns

correlation_matrix = df[numeric_columns].corr()

correlations_with_target = correlation_matrix[target_columns]

for target_column in target_columns:
    correlated_columns = correlations_with_target[target_column].sort_values(ascending=False)
    print(f"\nColumns most correlated with '{target_column}':")
    print(correlated_columns)

We then look for a good threshhold for what is considered a delay. 

In [ ]:
print("Decile 1:", df['ArrDelayMinutes'].quantile(0.10))
print("Decile 2:", df['ArrDelayMinutes'].quantile(0.20))
print("Decile 3:", df['ArrDelayMinutes'].quantile(0.30))
print("Decile 4:", df['ArrDelayMinutes'].quantile(0.40))
print("Decile 5:", df['ArrDelayMinutes'].quantile(0.50))
print("Decile 6:", df['ArrDelayMinutes'].quantile(0.60))
print("Decile 7:", df['ArrDelayMinutes'].quantile(0.70))
print("Decile 8:", df['ArrDelayMinutes'].quantile(0.80))
print("Decile 9:", df['ArrDelayMinutes'].quantile(0.90))
print("Decile 10:", df['ArrDelayMinutes'].quantile(1.0))

5 minute delay is a bit above the median so this will be our threshold for a delay. 
We then create a new Boolean column for whether the flight was delayed or not. 
Then we rewrite the DataFrame into a csv for convenience. 

In [ ]:
df['Delayed'] = df['ArrDelayMinutes'] >= 5
df.to_csv('PreWeatherML.csv', index=False, mode='w')

Lastly, we find some fun facts out of curiousity

In [ ]:
#Airline with the Most Delays:
most_delayed_airline = df.groupby('Airline')['ArrDelayMinutes'].mean().idxmax()
print(f"The airline with the most delays is {most_delayed_airline}.")

#Month with the Most Delays:
month_most_delays = df.groupby('Month')['ArrDelayMinutes'].mean().idxmax()
print(f"The month with the most delays is {month_most_delays}.")

#Year with the Most Delays:
year_most_delays = df.groupby('Year')['ArrDelayMinutes'].mean().idxmax()
print(f"The year with the most delays is {year_most_delays}.")

#Airport with the Longest Average Taxi Out Time:
airport_longest_taxi_out = df.groupby('OriginAirport')['TaxiOut'].mean().idxmax()
print(f"The airport with the longest average taxi out time is {airport_longest_taxi_out}.")

#Average Delay by City:
avg_arrival_delay_by_city = df.groupby('DestCityName')['ArrDelayMinutes'].mean().idxmax()
print(f"The city with the highest average arrival delay is {avg_arrival_delay_by_city}.")

#Busiest Airport (Most Flights):
busiest_airport = df['OriginAirport'].value_counts().idxmax()
print(f"The busiest airport (with the most flights) is {busiest_airport}.")

#Month with the Highest Average Airtime:
month_highest_airtime = df.groupby('Month')['AirTime'].mean().idxmax()
print(f"The month with the highest average airtime is {month_highest_airtime}.")

#Most Common Delay Type:
most_common_delay_type = df[['CarrierDelay', 'WeatherDelay', 'NASDelay', 'SecurityDelay', 'LateAircraftDelay']].idxmax(axis=1).mode().values[0]
print(f"The most common delay type is {most_common_delay_type}.")

#Day of the Week with the Fewest Delays:
day_fewest_delays = df.groupby('DayOfWeek')['ArrDelayMinutes'].mean().idxmin()
print(f"The day of the week with the fewest delays is {day_fewest_delays}.")


We now prepare for the machine learning step. Import in the necessary libraries 

In [ ]:
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error, r2_score

Weather gets read in here: Preprocessed into a MLRead file.

MLReady csv gets read in. Relevant features are defined as well as the variable that will be predicted. We are quantitatively predicting the delay time, so a random forest regression model will be used. 

In [ ]:
df = pd.read_csv('MLReady.csv')

selected_columns = ['Year', 'DayofMonth', 'Month', 'CRSDepTime', 'DepDelayMinutes',
                    'CRSArrTime', 'Distance', 'OriginSnow_Last2', 'OriginPrcp_Last2', 'OriginSnow',
                    'OriginPrcp', 'DestSnow_Last2', 'DestPrcp_Last2', 'DestSnow', 'DestPrcp',
                    'OriginAirport', 'DestAirport', 'DayOfWeek', 'Airline', 'ArrDelayMinutes']

#Excludes other columns
df = df[selected_columns]

We then define the categorical and numerical features of the model.
We create a list to store the evaluation metrics of each year.

In [ ]:
categorical_features = ['OriginAirport', 'DestAirport', 'DayOfWeek', 'Airline']
numerical_features = df.drop(columns=['ArrDelayMinutes']).columns.difference(categorical_features)

mae_list = []
r2_list = []

We transform the categorical variables by One Hot Encoding
We also define the scaler for the pipeline (Z-standardizing)
We define the preprocessor to deal with the cateogrical features and create the model

We want our model to train on year 2013 and test on 2014, then record its error, then train on 2014, and test on 2015 and so on. This will require a rolling regression model. The model then print its error for the current iteration, then after the for loop terminates, prints the average. 

In [ ]:
for year in range(2013, 2023):
    train_data = df[df['Year'] == year]
    test_data = df[df['Year'] == (year + 1)]

    X_train = train_data.drop(columns=['ArrDelayMinutes'])
    y_train = train_data['ArrDelayMinutes']

    X_test = test_data.drop(columns=['ArrDelayMinutes'])
    y_test = test_data['ArrDelayMinutes']

    numerical_transformer = StandardScaler()
    categorical_transformer = OneHotEncoder(handle_unknown='ignore', sparse=False)

    preprocessor = ColumnTransformer(
        transformers=[
            ('num', numerical_transformer, numerical_features),
            ('cat', categorical_transformer, categorical_features)
        ])

    model = Pipeline(steps=[('preprocessor', preprocessor),
                             ('regressor', RandomForestRegressor(n_estimators=10, random_state=42))])

    model.fit(X_train, y_train)

    y_pred = model.predict(X_test)

    mae = mean_absolute_error(y_test, y_pred)
    r2 = r2_score(y_test, y_pred)

    mae_list.append(mae)
    r2_list.append(r2)

    print(f'Training on {year}, Testing on {year + 1} - Mean Absolute Error: {mae}, R-squared: {r2}')

print(f'Average Mean Absolute Error: {sum(mae_list) / len(mae_list)}')
print(f'Average R-squared: {sum(r2_list) / len(r2_list)}')

In the model, "n_estimators = 10" can be increased for a more accurate depiction. However, due to the size of our data, n=10 will take 20+ minutes to load on an average computer so we figured this to be sufficient. 

As we can see, the results are pretty accurate. Now we will test this model on both Julia and my flight. Due to 2023 being an incomplete year, we will train on 2022. 

This will be the input for our flight back home. We will take this input and turn it into a separate DataFrame. 

In [ ]:
sean_flight_data = {
    'Year': 2023,
    'DayofMonth': 13,
    'Month': 12,
    'CRSDepTime': 1759,
    'DepDelayMinutes': 0,  # Replace with the actual delay for Sean's Flight
    'CRSArrTime': 2031,
    'Distance': 1946,
    'OriginSnow_Last2': 0,
    'OriginPrcp_Last2': 0,
    'OriginSnow': 0,
    'OriginPrcp': 0,
    'DestSnow_Last2': 0,
    'DestPrcp_Last2': 0,
    'DestSnow': 0,
    'DestPrcp': 0,
    'OriginAirport': 'Atlanta, GA: Hartsfield-Jackson Atlanta International',
    'DestAirport': 'Los Angeles, CA: Los Angeles International',
    'DayOfWeek': 'Wed',
    'Airline': 'Delta Air Lines Inc.'
}
sean_flight_df = pd.DataFrame([sean_flight_data])

julia_flight_data = {
    'Year': 2023,
    'DayofMonth': 17,
    'Month': 12,
    'CRSDepTime': 932,
    'DepDelayMinutes': 0,
    'CRSArrTime': 1315,
    'Distance': 541.79,
    'OriginSnow_Last2': 0,
    'OriginPrcp_Last2': 0,
    'OriginSnow': 0,
    'OriginPrcp': 0,
    'DestSnow_Last2': 0,
    'DestPrcp_Last2': 1955,
    'DestSnow': 0,
    'DestPrcp': 1955,
    'OriginAirport': 'Atlanta',
    'DestAirport': 'Washington, DC: Washington Dulles International',
    'DayOfWeek': 'Sun',
    'Airline': 'Frontier Airlines Inc.'
}
julia_flight_df = pd.DataFrame([julia_flight_data])

We then read in the machine learning DataFrame. We will use the same model as before.

In [ ]:
df = pd.read_csv('MLReady.csv')

selected_columns = ['Year', 'DayofMonth', 'Month', 'CRSDepTime', 'DepDelayMinutes',
                    'CRSArrTime', 'Distance', 'OriginSnow_Last2', 'OriginPrcp_Last2', 'OriginSnow',
                    'OriginPrcp', 'DestSnow_Last2', 'DestPrcp_Last2', 'DestSnow', 'DestPrcp',
                    'OriginAirport', 'DestAirport', 'DayOfWeek', 'Airline', 'ArrDelayMinutes']

df = df[selected_columns]

categorical_features = ['OriginAirport', 'DestAirport', 'DayOfWeek', 'Airline']
numerical_features = df.drop(columns=['ArrDelayMinutes']).columns.difference(categorical_features)

train_data = df[df['Year'] == 2022]

X_train = train_data.drop(columns=['ArrDelayMinutes'])
y_train = train_data['ArrDelayMinutes']

numerical_transformer = StandardScaler()
categorical_transformer = OneHotEncoder(handle_unknown='ignore', sparse_output=False)

preprocessor = ColumnTransformer(
    transformers=[
        ('num', numerical_transformer, numerical_features),
        ('cat', categorical_transformer, categorical_features)
    ])

model = Pipeline(steps=[('preprocessor', preprocessor),
                         ('regressor', RandomForestRegressor(n_estimators=10, random_state=42))])

model.fit(X_train, y_train)

julia_delay_prediction = model.predict(julia_flight_df)
print(f"Julia's Flight Delay Prediction: {julia_delay_prediction[0]} minutes")

sean_delay_prediction = model.predict(sean_flight_df)
print(f"Sean's Flight Delay Prediction: {sean_delay_prediction[0]} minutes")

Note to Julia, I was not sure where to put the following in our notebook. Lets discuss.

In [ ]:
# Years for training and testing
years_train = [2013, 2014, 2015, 2016, 2017, 2018, 2019, 2020, 2021, 2022]
mae_values = [6.487981144083558, 6.812233708127388, 5.903338926832098, 6.058833778954674, 6.42770809330232, 6.63172692814846, 4.971198419998488, 5.313820978632428, 6.2656328518457025, 7.222455108552838]
r_squared_values = [0.9132239974039758, 0.914890110438901, 0.9279673615797543, 0.9236424608313948, 0.9233149913249296, 0.9363628946806113, 0.916248103806017, 0.9451646019900752, 0.9516300949355766, 0.9558847170121954]

# Plot MAE
plt.figure(figsize=(10, 5))
plt.subplot(1, 2, 1)
plt.plot(years_train, mae_values, marker='o', linestyle='-', color='b')
plt.title('Mean Absolute Error (MAE)')
plt.xlabel('Training Year')
plt.ylabel('MAE')
plt.ylim(0, 10)  # Set y-axis limits

# Plot R-squared
plt.subplot(1, 2, 2)
plt.plot(years_train, r_squared_values, marker='o', linestyle='-', color='r')
plt.title('R-squared')
plt.xlabel('Training Year')
plt.ylabel('R-squared')
plt.ylim(0, 1)  # Set y-axis limits

plt.tight_layout()
plt.show()

We take the error values given to us by the model and plot them. Lastly, below, we graphed the mean delay per airline per year to notice any trends (Note to Julia:) This could be part of the exploratory analysis section. 

In [ ]:
df = pd.read_csv('MLready.csv')

grouped_df = df.groupby(['Year', 'Airline'])['ArrDelayMinutes'].mean().reset_index()

sns.set(style="whitegrid")
plt.figure(figsize=(12, 6))

sns.barplot(x='Year', y='ArrDelayMinutes', hue='Airline', data=grouped_df)

plt.title('Average Delays per Airline per Year')
plt.xlabel('Year')
plt.ylabel('Average Arrival Delay Minutes per Flight')

plt.show()